In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, classification_report

In [2]:
df1 = pd.read_csv("Datasets/instagram_reach.csv")
df1.sample()

,Unnamed: 0,S.No,USERNAME,Caption,Followers,Hashtags,Time since posted,Likes
30,3,11,greenhousegroup,"Onze Angela en Niek hosten vandaag NextM, hét ...",828,#nextmnl#groupm #greenhousegroup #eindhoven#ef...,2 hours,90


In [3]:
df1.reset_index(drop=True)
df1["Time"] = df1["Time since posted"].apply(lambda x: int(x.split(" ")[0]))
df1.drop(columns=["Unnamed: 0","S.No", "Caption", "USERNAME","Time since posted"], inplace=True)

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Followers  100 non-null    int64 
 1   Hashtags   100 non-null    object
 2   Likes      100 non-null    int64 
 3   Time       100 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 3.2+ KB


In [5]:
df1.describe()

,Followers,Likes,Time
count,100.00000,100.00000,100.000000
mean,961.96000,46.48000,3.460000
std,1014.62567,55.08698,3.394648
min,11.00000,8.00000,2.000000
25%,252.75000,19.00000,2.000000
50%,612.00000,29.00000,2.000000
75%,1197.00000,46.00000,3.000000
max,4496.00000,349.00000,24.000000


In [6]:
print(df1.loc[:0]["Hashtags"])

0    #MachineLearning #AI #DataAnalytics #DataScien...
Name: Hashtags, dtype: object


In [7]:
def convert_hastag(text):
    ps = PorterStemmer()
    wordnet = WordNetLemmatizer()
    
    tags = nltk.sent_tokenize(text)
    corpus = []
    tag = re.sub("[^a-zA-Z]", " ", text)
    tag = tag.lower()
    tag = tag.split()
#     tag = [wordnet.lemmatize(word) for word in tag if not word in set(stopwords.words("english"))]
    
    return tag

convert_hastag(df1["Hashtags"][0])

['machinelearning', 'ai', 'dataanalytics', 'datascienc', 'datalake']

In [8]:
hastag = pd.DataFrame()
df1["Hashtags"]= df1["Hashtags"].apply(convert_hastag)

In [9]:
print(df1[["Hashtags"]])

                                             Hashtags
0   [machinelearning, ai, dataanalytics, datascien...
1   [deck, mac, macintosh, sayhello, apple, stevej...
2   [whoiswho, aitrading, ai, aitradingteam, insta...
3   [iot, cre, workplace, cdo, bigdata, technology...
4   [instamachinelearning, instabigdata, instamark...
..                                                ...
95  [beverlyhills, realestate, losangelesrealestat...
96  [workspace, work, developer, development, deve...
97  [books, book, motivation, inspiration, life, b...
98  [heavyequipment, underconstruction, dozer, rea...
99  [marketing, programming, development, desarrol...

[100 rows x 1 columns]


In [10]:
unique_hashtag = set(tag for row in df1["Hashtags"] for tag in row)
len(unique_hashtag)

1155

In [11]:
for hashtag in unique_hashtag:
    df1[hashtag] = 0 # initialize 0 value for every hashtag


In [12]:
for index, row in enumerate(df1["Hashtags"]): # enumerate hashtags 
    for tag in row: # Row wise hashtag 
        df1.at[index, tag] = 1 # appply 1 value when index and tag match 


In [13]:
df1.drop(columns=["Hashtags"], inplace=True)

In [14]:
x1 = df1.drop(columns=["Likes", "Time"])

In [15]:
y1 = df1[["Likes", "Time"]]

In [16]:
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size=0.1)

In [17]:
print(f"X_train shape = {x1_train.shape}")
print(f"X_test shape = {x1_test.shape}")
print(f"y_train shape = {y1_train.shape}")
print(f"y_test shape = {y1_test.shape}")

X_train shape = (90, 1156)
X_test shape = (10, 1156)
y_train shape = (90, 2)
y_test shape = (10, 2)


In [18]:
y1_test

,Likes,Time
18,30,4
91,46,2
0,139,11
65,29,2
95,31,3
25,46,2
69,40,2
20,198,5
14,28,3
38,18,2


In [19]:
rf = RandomForestRegressor()

In [20]:
rf.fit(x1_train, y1_train)

RandomForestRegressor()

In [21]:
y1_pred = rf.predict(x1_test)
y1_pred

array([[19.4       ,  2.87      ],
       [36.68      ,  3.11      ],
       [63.87      ,  6.56      ],
       [38.22      ,  3.59      ],
       [36.94      ,  2.24      ],
       [25.07      ,  2.53      ],
       [22.49      ,  2.23      ],
       [34.89      ,  3.08      ],
       [56.13      ,  5.03      ],
       [24.245     ,  2.23833333]])

In [22]:
mse1 = mean_squared_error(y1_test, y1_pred)
mse1

1708.8695763888893

In [23]:
rmse1 = np.sqrt(mse1)
rmse1

41.33847573857664

In [24]:
r2score1 = r2_score(y1_test, y1_pred)
r2score1

0.22102912055366447

In [25]:
df2 = pd.read_csv("DataSets/ObesityDataSet_raw_and_data_sinthetic.csv")

In [26]:
df2.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [27]:
df2.isnull().sum()

Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [28]:
df2.describe()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.685628,2.008011,1.010298,0.657866
std,6.345968,0.093305,26.191172,0.533927,0.778039,0.612953,0.850592,0.608927
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,19.947192,1.630000,65.473343,2.000000,2.658738,1.584812,0.124505,0.000000
50%,22.777890,1.700499,83.000000,2.385502,3.000000,2.000000,1.000000,0.625350
75%,26.000000,1.768464,107.430682,3.000000,3.000000,2.477420,1.666678,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000


In [29]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [30]:
df2.sample(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
1402,Female,37.441044,1.524293,76.202761,yes,yes,2.000000,2.994046,Sometimes,no,1.603549,no,1.335857,0.000000,Sometimes,Automobile,Obesity_Type_I
1566,Male,29.429687,1.910987,129.935666,yes,yes,2.200588,3.000000,Sometimes,no,1.295697,no,1.058378,0.000000,Sometimes,Public_Transportation,Obesity_Type_II
897,Male,26.703710,1.802871,85.770013,yes,yes,2.872121,3.051804,Sometimes,no,2.111913,no,0.843709,0.153559,Sometimes,Automobile,Overweight_Level_I
1460,Male,23.000000,1.742500,105.028665,yes,yes,2.393837,3.000000,Sometimes,no,2.014990,no,0.978815,0.413220,Sometimes,Public_Transportation,Obesity_Type_I
482,Female,20.000000,1.580000,53.500000,yes,yes,2.000000,1.000000,Frequently,no,2.000000,no,1.000000,1.000000,Sometimes,Public_Transportation,Normal_Weight


In [31]:
df2["SCC"].unique()

array(['no', 'yes'], dtype=object)

In [32]:
df2[["Age", "Weight"]] = df2[["Age", "Weight"]].astype(int)

In [33]:
df2["Gender"] = df2["Gender"].apply(lambda x : 1 if x == "Male" else 0)

In [34]:
df2["SMOKE"] = df2["SMOKE"].apply(lambda x : 1 if x == "yes" else 0)

In [35]:
df2["family_history_with_overweight"] = df2["family_history_with_overweight"].apply(lambda x : 1 if x == "yes" else 0)

In [36]:
df2["FAVC"] = df2["FAVC"].apply(lambda x : 1 if x == "yes" else 0)

In [37]:
df2["SCC"] = df2["SCC"].apply(lambda x : 1 if x == "yes" else 0)

In [38]:
df2

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,21,1.620000,64,1,0,2.0,3.0,Sometimes,0,2.000000,0,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,0,21,1.520000,56,1,0,3.0,3.0,Sometimes,1,3.000000,1,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,1,23,1.800000,77,1,0,2.0,3.0,Sometimes,0,2.000000,0,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,1,27,1.800000,87,0,0,3.0,3.0,Sometimes,0,2.000000,0,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,1,22,1.780000,89,0,0,2.0,1.0,Sometimes,0,2.000000,0,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0,20,1.710730,131,1,1,3.0,3.0,Sometimes,0,1.728139,0,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,0,21,1.748584,133,1,1,3.0,3.0,Sometimes,0,2.005130,0,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,0,22,1.752206,133,1,1,3.0,3.0,Sometimes,0,2.054193,0,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,0,24,1.739450,133,1,1,3.0,3.0,Sometimes,0,2.852339,0,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [39]:
df2["MTRANS"].unique()

array(['Public_Transportation', 'Walking', 'Automobile', 'Motorbike',
       'Bike'], dtype=object)

In [40]:
label_encoder = LabelEncoder()

In [41]:
df2["CAEC"] =label_encoder.fit_transform(df2["CAEC"])

In [42]:
df2["CALC"] = label_encoder.fit_transform(df2["CALC"])

In [43]:
df2["MTRANS"] = label_encoder.fit_transform(df2["MTRANS"])

In [44]:
df2["NObeyesdad"] = label_encoder.fit_transform(df2["NObeyesdad"])

In [45]:
df2.sample(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
1929,0,25,1.663632,105,1,1,3.000000,3.000000,2,0,1.626467,0,0.010183,0.412806,2,3,4
1576,1,26,1.780731,112,1,1,2.263245,3.000000,2,0,2.092509,0,0.000000,1.836853,2,0,3
38,1,23,1.650000,67,1,1,2.000000,3.000000,2,0,2.000000,0,1.000000,1.000000,2,3,1
664,0,21,1.689441,51,1,1,3.000000,3.715118,1,0,1.774576,0,0.102970,0.646423,3,3,0
502,0,21,1.843419,165,1,1,3.000000,3.000000,2,0,2.406541,0,0.100320,0.479221,2,3,4


In [46]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   int64  
 1   Age                             2111 non-null   int32  
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   int32  
 4   family_history_with_overweight  2111 non-null   int64  
 5   FAVC                            2111 non-null   int64  
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   int32  
 9   SMOKE                           2111 non-null   int64  
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   int64  
 12  FAF                             21

# 4 Different Classification algorithms

In [47]:
x2 = df2.drop(columns=["NObeyesdad"])
y2 = df2["NObeyesdad"]

In [48]:
y2

0       1
1       1
2       1
3       5
4       6
       ..
2106    4
2107    4
2108    4
2109    4
2110    4
Name: NObeyesdad, Length: 2111, dtype: int32

In [49]:
x2_train, x2_test, y2_train, y2_test = train_test_split(x2,y2, test_size=0.2)

In [50]:
log_r2 = LogisticRegression(solver="liblinear")

In [51]:
log_r2.fit(x2_train, y2_train)

LogisticRegression(solver='liblinear')

In [52]:
y2_pred_log = log_r2.predict(x2_test)

In [53]:
accuracy_score(y2_test, y2_pred_log)

0.7257683215130024

In [54]:
knn2 = KNeighborsClassifier()

In [55]:
knn2.fit(x2_train, y2_train)

KNeighborsClassifier()

In [56]:
y2_pred_knn = knn2.predict(x2_test)

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:189: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [57]:
accuracy_score(y2_test, y2_pred_knn)

0.8865248226950354

In [58]:
svm2 = svm.SVC(kernel="linear")

In [59]:
svm2.fit(x2_train, y2_train)

SVC(kernel='linear')

In [60]:
y2_pred_svm = svm2.predict(x2_test)

In [61]:
accuracy_score(y2_test, y2_pred_svm)

0.8628841607565012

In [62]:
naive2 = MultinomialNB()

In [63]:
naive2.fit(x2_train, y2_train)

MultinomialNB()

In [64]:
y2_pred_naive = naive2.predict(x2_test)

In [65]:
accuracy_score(y2_test, y2_pred_naive)

0.5791962174940898

In [67]:
df_3 =  pd.read_json("DataSets/News_Category_Dataset_v3.json", lines=True)

In [73]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  object        
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.6+ MB


In [84]:
df_3.sample(5)

,link,headline,category,short_description,authors,date
118831,https://www.huffingtonpost.com/entry/movie-loc...,50 Fictional Places You Can Visit In Real Life,TRAVEL,,Suzy Strutner,2014-10-05
121854,https://www.huffingtonpost.com/entry/when-kids...,When Kids Feel 'Fat',HEALTHY LIVING,I see 7-year-old kids who agonize over what to...,"Andrea Wachter, ContributorPsychotherapist and...",2014-09-01
114639,https://www.huffingtonpost.com/entry/10-easy-r...,10 Easy Recipes for a Stress-Free Thanksgiving,TASTE,I used to host elaborate Thanksgiving celebrat...,"Jennifer Segal, ContributorChef, Recipe Develo...",2014-11-23
102795,https://www.huffingtonpost.com/entry/music-i-m...,Music I (Mostly) Hold Dear: String Quartets of...,ARTS,The String Quartet was initiated by Joseph Hay...,"Daniel Asia, ContributorComposer; Professor, U...",2015-04-09
159168,https://www.huffingtonpost.com/entry/evan-will...,"Evan Williams, Twitter Cofounder, Lists Surpri...",HOME & LIVING,"Internet wealth does strange things to people,...",Robin Wilkey,2013-07-19


In [81]:
df_3["category"].nunique(), df_3["authors"].nunique()

(42, 29169)

In [85]:
vectoriser = CountVectorizer(stop_words="english")

In [106]:
df_3.iloc[0]["headline"]

'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters'

In [107]:
re.sub(re.compile("<.*?>"),'', df_3.iloc[0]["headline"]) 

'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters'

In [102]:
vectoriser.fit_transform([df_3.iloc[0]["headline"].replace("/"," ").replace("-"," ")]).toarray()

array([[1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)